# Imports

In [ ]:
import sys
from typing import Optional, Union, List

parentdir = '/home/jerzy/Documents/GitHub/jrzkaminski/FEDOT/'
bamtdir = '/home/jerzy/Documents/GitHub/jrzkaminski/BAMT'
sys.path.insert(0, parentdir)
sys.path.insert(0, bamtdir)

from math import ceil
from pgmpy.estimators import K2Score
from pgmpy.models import BayesianNetwork
from fedot.core.pipelines.convert import graph_structure_as_nx_graph
from fedot.core.optimisers.optimizer import GraphGenerationParams
from fedot.core.optimisers.graph import OptGraph, OptNode
from fedot.core.optimisers.objective.objective_eval import ObjectiveEvaluate
from fedot.core.optimisers.objective.objective import Objective
from fedot.core.optimisers.gp_comp.operators.selection import SelectionTypesEnum
from fedot.core.optimisers.gp_comp.gp_optimiser import EvoGraphOptimiser, GPGraphOptimiserParameters, \
    GeneticSchemeTypesEnum
from fedot.core.optimisers.adapters import DirectAdapter
from fedot.core.dag.verification_rules import has_no_cycle, has_no_self_cycled_nodes
from examples.divided_bn import DividedBN
from fedot.core.composer.gp_composer.gp_composer import PipelineComposerRequirements
import bamt.preprocessors as pp
import bamt.networks as Nets
from sklearn import preprocessing
import random
import pandas as pd
import numpy as np
from copy import deepcopy
from fedot.core.dag.graph import Graph
import time

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
pigs = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/pigs.csv')
win95pts = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/win95pts.csv')
hailfinder = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/hailfinder.csv')
hepar2 = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/hepar2.csv')
arth150 = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/arth150.csv')
ecoli70 = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/ecoli70.csv', index_col=0)
magic_irri = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/magic-irri.csv', index_col=0)
magic_niab = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/magic-niab.csv', index_col=0)

pigs_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/pigs_true.csv')
win95pts_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/win95pts_true.csv')
hailfinder_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/hailfinder_true.csv')
hepar2_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/hepar2_true.csv')
arth150_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/arth150_true.csv')
ecoli70_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/ecoli70_true.csv')
magic_irri_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/magic-irri_true.csv')
magic_niab_true = pd.read_csv('https://raw.githubusercontent.com/jrzkaminski/BAMT-old/main/data/magic-niab_true.csv')

# Featured functions

In [ ]:
class CustomGraphModel(Graph):

    def __init__(self, nodes: Optional[Union[OptNode, List[OptNode]]] = None):
        super().__init__(nodes)
        self.unique_pipeline_id = 1


class CustomGraphNode(OptNode):
    def __str__(self):
        return self.content["name"]


def child_dict(net: list):
    res_dict = dict()
    for e0, e1 in net:
        if e1 in res_dict:
            res_dict[e1].append(e0)
        else:
            res_dict[e1] = [e0]
    return res_dict

def precision_recall(pred_net: list, true_net: list, decimal = 2):
    pred_dict = child_dict(pred_net)
    true_dict = child_dict(true_net)
    corr_undir = 0
    corr_dir = 0
    for e0, e1 in pred_net:
        flag = True
        if e1 in true_dict:
            if e0 in true_dict[e1]:
                corr_undir += 1
                corr_dir += 1
                flag = False
        if (e0 in true_dict) and flag:
            if e1 in true_dict[e0]:
                corr_undir += 1
    pred_len = len(pred_net)
    true_len = len(true_net)
    shd = pred_len + true_len - corr_undir - corr_dir
    return {'AP': round(corr_undir/pred_len, decimal), 'AR': round(corr_undir/true_len, decimal), 'AHP': round(corr_dir/pred_len, decimal), 'AHR': round(corr_dir/true_len, decimal), 'SHD': shd}


# задаем метрику
def custom_metric(graph: CustomGraphModel, data: pd.DataFrame):
    graph_nx, labels = graph_structure_as_nx_graph(graph)
    struct = []
    for meta_edge in graph_nx.edges():
        l1 = str(labels[meta_edge[0]])
        l2 = str(labels[meta_edge[1]])
        struct.append([l1, l2])

    bn_model = BayesianNetwork(struct)
    bn_model.add_nodes_from(data.columns)

    score = K2Score(data).score(bn_model)
    return [-score]


# задаем кроссовер (обмен ребрами)
def custom_crossover_exchange_edges(graph_first: OptGraph, graph_second: OptGraph, max_depth):
    def find_node(graph: OptGraph, node):
        return graph.nodes[dir_of_nodes[node.content['name']]]

    num_cros = 100
    try:
        for _ in range(num_cros):
            new_graph_first = deepcopy(graph_first)
            new_graph_second = deepcopy(graph_second)

            edges_1 = new_graph_first.operator.get_all_edges()
            edges_2 = new_graph_second.operator.get_all_edges()
            count = ceil(min(len(edges_1), len(edges_2)) / 2)
            choice_edges_1 = random.sample(edges_1, count)
            choice_edges_2 = random.sample(edges_2, count)

            for meta_edge in choice_edges_1:
                new_graph_first.operator.disconnect_nodes(meta_edge[0], meta_edge[1], False)
            for meta_edge in choice_edges_2:
                new_graph_second.operator.disconnect_nodes(meta_edge[0], meta_edge[1], False)

            old_edges1 = new_graph_first.operator.get_all_edges()
            old_edges2 = new_graph_second.operator.get_all_edges()

            new_edges_2 = [[find_node(new_graph_second, i[0]), find_node(new_graph_second, i[1])]
                           for i in choice_edges_1]
            new_edges_1 = [[find_node(new_graph_first, i[0]), find_node(new_graph_first, i[1])] for i in choice_edges_2]
            for meta_edge in new_edges_1:
                if meta_edge not in old_edges1:
                    new_graph_first.operator.connect_nodes(meta_edge[0], meta_edge[1])
            for meta_edge in new_edges_2:
                if meta_edge not in old_edges2:
                    new_graph_second.operator.connect_nodes(meta_edge[0], meta_edge[1])

            if old_edges1 != new_graph_first.operator.get_all_edges() or old_edges2 != new_graph_second.operator.get_all_edges():
                break

        if old_edges1 == new_graph_first.operator.get_all_edges() and new_edges_1 != [] and new_edges_1 != None:
            new_graph_first = deepcopy(graph_first)
        if old_edges2 == new_graph_second.operator.get_all_edges() and new_edges_2 != [] and new_edges_2 != None:
            new_graph_second = deepcopy(graph_second)
    except Exception as ex:
        print(ex)
    return new_graph_first, new_graph_second


# задаем три варианта мутации: добавление узла, удаление узла, разворот узла
def custom_mutation_add(graph: OptGraph, **kwargs):
    num_mut = 100
    try:
        for _ in range(num_mut):
            rid = random.choice(range(len(graph.nodes)))
            random_node = graph.nodes[rid]
            other_random_node = graph.nodes[random.choice(range(len(graph.nodes)))]
            nodes_not_cycling = (random_node.descriptive_id not in
                                 [n.descriptive_id for n in other_random_node.ordered_subnodes_hierarchy()] and
                                 other_random_node.descriptive_id not in
                                 [n.descriptive_id for n in random_node.ordered_subnodes_hierarchy()])
            if nodes_not_cycling:
                graph.operator.connect_nodes(random_node, other_random_node)
                break

    except Exception as ex:
        graph.log.warn(f'Incorrect connection: {ex}')
    return graph


def custom_mutation_delete(graph: OptGraph, **kwargs):
    num_mut = 100
    try:
        for _ in range(num_mut):
            rid = random.choice(range(len(graph.nodes)))
            random_node = graph.nodes[rid]
            other_random_node = graph.nodes[random.choice(range(len(graph.nodes)))]
            if random_node.nodes_from is not None and other_random_node in random_node.nodes_from:
                graph.operator.disconnect_nodes(other_random_node, random_node, False)
                break
    except Exception as ex:
        print(ex)
    return graph


def custom_mutation_reverse(graph: OptGraph, **kwargs):
    num_mut = 100
    try:
        for _ in range(num_mut):
            rid = random.choice(range(len(graph.nodes)))
            random_node = graph.nodes[rid]
            other_random_node = graph.nodes[random.choice(range(len(graph.nodes)))]
            if random_node.nodes_from is not None and other_random_node in random_node.nodes_from:
                graph.operator.reverse_edge(other_random_node, random_node)
                break
    except Exception as ex:
        print(ex)
    return graph


# задаем правила на запрет дублирующих узлов
def _has_no_duplicates(graph):
    _, labels = graph_structure_as_nx_graph(graph)
    if len(labels.values()) != len(set(labels.values())):
        raise ValueError('Custom graph has duplicates')
    return True

# Algorithm

In [ ]:
def get_edges_by_localstructures(data, datatype="discrete", max_local_structures=8):

    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    global local_edges, root_nodes, child_nodes, initial_df

    initial_df = data

    # initialize divided_bn

    start_time = time.time()

    divided_bn = DividedBN(data=data, max_local_structures=max_local_structures)

    divided_bn.set_local_structures(data, datatype=datatype)

    local_edges = divided_bn.local_structures_edges

    divided_bn.set_hidden_nodes(data=data)

    hidden_df = pd.DataFrame.from_dict(divided_bn.hidden_nodes)

    hidden_df.columns = hidden_df.columns.astype(str)

    root_nodes = divided_bn.root_nodes
    child_nodes = divided_bn.child_nodes

    vertices = list(hidden_df.columns)

    encoder = preprocessing.LabelEncoder()
    discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
    p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
    discretized_data, _ = p.apply(hidden_df)

    # словарь: {имя_узла: уникальный_номер_узла}
    global dir_of_nodes
    dir_of_nodes = {hidden_df.columns[i]: i for i in range(len(hidden_df.columns))}

    # правила для байесовских сетей: нет петель, нет циклов, нет повторяющихся узлов
    rules = [has_no_self_cycled_nodes, has_no_cycle, _has_no_duplicates]

    # задаем для оптимизатора fitness-функцию
    objective = Objective(custom_metric)
    objective_eval = ObjectiveEvaluate(objective, data=discretized_data)
    # инициализация начальной сети (пустая)
    initial = [CustomGraphModel(nodes=[CustomGraphNode(nodes_from=None,
                                                       content={'name': vertex}) for vertex in vertices])]

    requirements = PipelineComposerRequirements(
        primary=vertices,
        secondary=vertices,
        max_arity=100,
        max_depth=100,
        pop_size=15,
        num_of_generations=50,
        crossover_prob=0.8,
        mutation_prob=0.9
    )

    optimiser_parameters = GPGraphOptimiserParameters(
        genetic_scheme_type=GeneticSchemeTypesEnum.steady_state,
        selection_types=[SelectionTypesEnum.tournament],
        mutation_types=[custom_mutation_add, custom_mutation_delete, custom_mutation_reverse],
        crossover_types=[custom_crossover_exchange_edges]
    )

    graph_generation_params = GraphGenerationParams(
        adapter=DirectAdapter(base_graph_class=CustomGraphModel, base_node_class=CustomGraphNode),
        rules_for_constraint=rules)

    optimiser = EvoGraphOptimiser(
        graph_generation_params=graph_generation_params,
        parameters=optimiser_parameters,
        requirements=requirements,
        initial_graph=initial,
        objective=objective)

    # запуск оптимизатора
    optimized_graph = optimiser.optimise(objective_eval)[0]
    # вывод полученного графа

    print("--- %s seconds ---" % (time.time() - start_time))

    evolutionary_edges = optimized_graph.operator.get_all_edges()

    # print('evolutionary_edges', evolutionary_edges)

    # optimized_graph.show()

    local_edges_merged = []

    for key in local_edges:
         local_edges_merged += local_edges[key]

    # print('local_edges_merged', local_edges_merged)

    external_edges = []

    for meta_edge in evolutionary_edges:
        for root_node in root_nodes[int(str(meta_edge[0]))]:
            for child_node in child_nodes[int(str(meta_edge[1]))]:
                external_edges.append([root_node, child_node])


    # unpacked_hidden_edges = []
    # unpacked_hidden_edges = np.array(unpacked_hidden_edges)

    # for meta_edge in evolutionary_edges:
    #     local_root_nodes = root_nodes[int(str(meta_edge[0]))]
    #     local_child_nodes = child_nodes[int(str(meta_edge[1]))]
    #     for root_node in local_root_nodes:
    #         for child_node in local_child_nodes:
    #             unpacked_hidden_edges = np.append(unpacked_hidden_edges, [[root_node, child_node]], axis=0)

    # print('Unpacked hidden edges:', unpacked_hidden_edges)

    all_edges = local_edges_merged + external_edges

    return all_edges

# Experiments

In [ ]:
def f1_score(custom_dag_edges, reference_dag_edges):
    # Calculate True Positives (tp), False Positives (fp), and False Negatives (fn)
    tp = fp = fn = 0
    for edge in custom_dag_edges:
        if edge in reference_dag_edges:
            tp += 1
        else:
            fp += 1

    for edge in reference_dag_edges:
        if edge not in custom_dag_edges:
            fn += 1

    # Calculate Precision and Recall
    if tp + fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)

    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)

    # Calculate F1 Score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return f1

def convert_to_string(nested_list):
    return [[str(item) for item in inner_list] for inner_list in nested_list]


In [ ]:
pigs_evo = get_edges_by_localstructures(pigs, datatype='discrete', max_local_structures=16)


In [ ]:
precision_recall(convert_to_string(pigs_evo), pigs_true.values.tolist())['SHD']


In [ ]:
f1_score(convert_to_string(pigs_evo), pigs_true.values.tolist())

In [ ]:
win95pts_evo = get_edges_by_localstructures(win95pts, max_local_structures=4)


In [ ]:
precision_recall(win95pts_true.values.tolist(), convert_to_string(win95pts_evo))['SHD']

In [ ]:
f1_score(win95pts_true.values.tolist(), convert_to_string(win95pts_evo))

In [ ]:
hailfinder_evo = get_edges_by_localstructures(hailfinder, max_local_structures=8)


In [ ]:
precision_recall(hailfinder_true.values.tolist(), convert_to_string(hailfinder_evo))['SHD']

In [ ]:
f1_score(hailfinder_true.values.tolist(), convert_to_string(hailfinder_evo))

In [ ]:
hepar2_evo = get_edges_by_localstructures(hepar2, max_local_structures=4)


In [ ]:
precision_recall(hepar2_true.values.tolist(), hepar2_evo)['SHD']

In [ ]:
f1_score(hepar2_true.values.tolist(), hepar2_evo)

In [ ]:
arth150_evo = get_edges_by_localstructures(arth150, datatype='continuous', max_local_structures=8)


In [ ]:
precision_recall(arth150_true.values.tolist(), arth150_evo)['SHD']

In [ ]:
ecoli70_evo = get_edges_by_localstructures(ecoli70, datatype='continuous', max_local_structures=4)


In [ ]:
precision_recall(ecoli70_true.values.tolist(), ecoli70_evo)['SHD']

In [ ]:
magic_irri_evo = get_edges_by_localstructures(magic_irri, datatype='continuous', max_local_structures=4)


In [ ]:
precision_recall(magic_irri_true.values.tolist(), magic_irri_evo)['SHD']

In [ ]:
magic_niab_evo = get_edges_by_localstructures(magic_niab, datatype='continuous', max_local_structures=4)

In [ ]:
precision_recall(magic_niab_true.values.tolist(), magic_niab_evo)['SHD']


In [ ]:
stat = pd.read_csv('/Users/jerzypro/Documents/GitHub/FEDOT/examples/pres.csv')


In [ ]:
stat

In [ ]:
stat = stat.iloc[[5,6,8,9,10,11,12,13]]
stat

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.rcParams.update({'font.size': 22})
stat.plot(x="dataset", y=["shd_bidag", "shd_brave", "shd_ga_hidden"], kind="bar",figsize=(20,10))
plt.xlabel('Dataset', fontdict={'fontsize': 22})
plt.ylabel('SHD', fontdict={'fontsize': 22})
plt.title('Structural hamming distance for different benchmark datasets', fontdict={'fontsize': 20})
plt.show()

In [ ]:
plt.figure()
plt.rcParams.update({'font.size': 22})
stat.plot(x="dataset", y=["time_bidag", "time_brave", "time_ga_hidden"], kind="bar",figsize=(20,10))
plt.xlabel('Dataset', fontdict={'fontsize': 22})
plt.ylabel('time, s', fontdict={'fontsize': 22})
plt.title('Time for different benchmark datasets', fontdict={'fontsize': 20})
plt.show()

# Complexity analysis

In [ ]:
df = pigs.copy()

In [ ]:
import time


# define the range of variable counts you want to test
variable_counts = range(100, 200, 20)

# create an empty list to store the results
results = []

# loop through the variable counts and time the execution of your function
for count in variable_counts:
    # select the first `count` columns from your DataFrame
    data = df.iloc[:, :count]
    # calculate the maximum local structures based on the selected number of variables
    max_local_structures = count // 25
    # run the function 5 times and record the durations
    durations = []
    for i in range(3):
        start_time = time.time()
        get_edges_by_localstructures(data, datatype='discrete', max_local_structures=max_local_structures)
        duration = time.time() - start_time
        durations.append(duration)
    # take the average duration of the 5 runs and store the result in a dictionary
    result = {'variables': count, 'duration': sum(durations) / len(durations)}
    results.append(result)


In [ ]:
reults_amd_5800x3d = [{'variables': 100, 'duration': 39.65796987215678},
 {'variables': 110, 'duration': 49.2115273475647},
 {'variables': 120, 'duration': 57.53783130645752},
 {'variables': 130, 'duration': 61.85720213254293},
 {'variables': 140, 'duration': 59.63492830594381},
 {'variables': 150, 'duration': 77.87671264012654},
 {'variables': 160, 'duration': 87.72854121526082},
 {'variables': 170, 'duration': 87.38114698727925},
 {'variables': 180, 'duration': 90.07503477732341},
 {'variables': 190, 'duration': 97.80088567733765},
 {'variables': 200, 'duration': 92.55843989054362},
 {'variables': 210, 'duration': 112.06234796841939},
 {'variables': 220, 'duration': 119.9249399503072},
 {'variables': 230, 'duration': 133.53242739041647},
 {'variables': 240, 'duration': 129.52960761388144},
 {'variables': 250, 'duration': 133.55799412727356},
 {'variables': 260, 'duration': 146.34153779347739},
 {'variables': 270, 'duration': 166.4611929257711},
 {'variables': 280, 'duration': 165.3845240275065},
 {'variables': 290, 'duration': 143.90822863578796},
 {'variables': 300, 'duration': 164.4334537188212},
 {'variables': 310, 'duration': 208.56076248486838},
 {'variables': 320, 'duration': 198.76508418718973},
 {'variables': 330, 'duration': 204.2042313416799},
 {'variables': 340, 'duration': 196.72765175501505},
 {'variables': 350, 'duration': 208.47475425402322},
 {'variables': 360, 'duration': 209.39110573132834},
 {'variables': 370, 'duration': 245.15379524230957},
 {'variables': 380, 'duration': 249.4655213356018},
 {'variables': 390, 'duration': 253.62290811538696},
 {'variables': 400, 'duration': 274.08744796117145},
 {'variables': 410, 'duration': 214.6125543912252},
 {'variables': 420, 'duration': 236.08180085817972},
 {'variables': 430, 'duration': 241.00591564178467},
 {'variables': 440, 'duration': 241.12759121259054}]
 

results_m1_pro_8core = [
    
]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# create lists of x and y values for the plot
x_values = [result['variables'] for result in reults_amd_5800x3d]
y_values = [result['duration'] for result in reults_amd_5800x3d]

# fit a polynomial curve to the data
poly_degree = 2  # degree of the polynomial regression
poly_features = PolynomialFeatures(degree=poly_degree)
x_poly = poly_features.fit_transform(np.array(x_values).reshape(-1, 1))
poly_model = LinearRegression()
poly_model.fit(x_poly, y_values)
poly_y_pred = poly_model.predict(x_poly)

# create the plot
plt.scatter(x_values, y_values)
plt.plot(x_values, poly_y_pred, color='red')
plt.title('Execution Time vs. Number of Variables, Ryzen 5800x3d')
plt.xlabel('Number of Variables')
plt.ylabel('Execution Time (s)')
plt.show()

# estimate the time complexity of the algorithm
coef = poly_model.coef_
complexity_str = 'O('
for i in range(len(coef)):
    if i == 0:
        complexity_str += str(round(coef[i], 2))
    else:
        complexity_str += f'+{str(round(coef[i], 2))}*n^{str(i)}'
complexity_str += ')'
print(f"Estimated time complexity: {complexity_str}")
